<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Метод Рунге-Кутты

### Modules - Ordinary Differential Equations
<section class="post-meta">
By Magnus H-S Dahle, Henning G. Hugdal, Håkon W. Ånes and Peter Berg
</section>
Last edited: February 7th 2018 
___

В других блокнотах мы изучали метод Эйлера для решения обыкновенных дифференциальных уравнений (ОДУ). Этот метод, пожалуй, самый простой для понимания и реализации и поэтому очень популярен в качестве учебного материала. Однако для научных целей метод Эйлера неточен по сравнению с более сложными методами, использующими тот же размер шага, а также условно неустойчив, как обсуждалось в предыдущих модулях.

Здесь мы рассмотрим метод Рунге-Кутты, который является одним из наиболее применяемых численных методов для решения ОДУ. Он считается достаточно эффективным, учитывая время вычислений, и его аппроксимация четвертого порядка обеспечивает приличную точность, то есть достаточно хороша для большинства задач, не требующих решений с очень высокой точностью. Обратите внимание, что мы представляем метод в его простейшей форме: вы можете столкнуться с его реализациями в другом виде. В целом, новые (и более интеллектуальные) реализации метода могут значительно повысить его эффективность.

## Пересмотр метода Эйлера

Рассмотрим то же самое (нелинейное) ОДУ, которое обсуждалось в предыдущих модулях,

$$
\dot{x}(t) = \cos(x(t)) + \sin(t), \qquad \dot{x}\equiv\frac{\textrm{d}x(t)}{\textrm{d}t}
$$

с начальным условием $x(t_0) = 0, t_0 = 0$. (Явный) метод Эйлера решает эту проблему путем дискретизации переменных таким образом, что 

\begin{align*}
t     & \rightarrow t_n     \qquad\equiv t_0 + n\cdot\Delta t,\\[1.0em]
x(t)  & \rightarrow x(t_n)  \quad\equiv x_n                 ,\\[1.0em]
n     & = 0,1,\ldots,N;                 
\end{align*}

Затем значение функции $x_{n+1}$ может быть аппроксимировано предыдущим значением плюс изменение (производная) в этой точке, умноженное на расстояние во времени $\Delta t$ между $x_n$ и $x_{n+1}$. То есть,

$$
x_{n+1} = x_n + (\Delta t) \cdot \dot{x}_n, \qquad \Delta t = \frac{t_N - t_0}{N} \equiv h
$$

Для этой конкретной задачи нам дается $\dot{x}_n(t_n)$, так что выражение может быть подставлено непосредственно в формулу Эйлера, дающую,

$$
x_{n+1} = x_n + h [\cdot \cos(x_n) + \sin(t_n)]
$$

Это примерно все, что нужно для решения текущей проблемы. 

In [ ]:
%matplotlib inline
import numpy as np               # Numerical Python
import matplotlib.pyplot as plt  # Graph and Plot handling
import time                      # Time Measure

In [ ]:
# Быстрая реализация метода Эйлера для решения
# вышеуказанной задачи с начальным значением

# Попробуйте настроить "N", чтобы увидеть, как сходится численное решение

t0   = 0.0
tN   = 10.0
N    = 15
t    = np.linspace(t0,tN,N)
x_Eu = np.zeros(N) ## Euler
h    = (tN-t0)/N

# Начальное условие:
x_Eu[0] = 0

for n in range(0,N-1):
    x_Eu[n+1] = x_Eu[n] + h * (np.cos(x_Eu[n]) + np.sin(t[n]))

plt.figure(figsize=(10,6))
plt.plot(t,x_Eu,'-ro',linewidth=3.0, label=r'Euler')
plt.ylabel(r'Dimensionless position, $x(t)$')
plt.xlabel(r'Dimensionless time, $t$')
plt.legend(loc=4) # 'loc' задает расположение текста легенды в сюжете 
plt.grid()
plt.show()

Быстрый анализ ошибок показывает, что локальная ошибка усечения, возникающая в <i>на каждом шаге</i> метода Эйлера, пропорциональна размеру шага в квадрате $h^2 = (\Delta t)^2$, если Тейлор расширит функцию примерно на $t+h$

\begin{align*}
\text{Exact:} & \qquad x(t + h) =  x(t) + h\dot{x} + \frac{h^2}{2}\ddot{x} + \frac{h^3}{6}\dddot{x} + \ldots  \\[1.2em]
\text{Euler:} & \qquad x(t + h) \approx  x(t) + h\dot{x}\\[1.2em]
\text{Error:} & \qquad e = \frac{h^2}{2}\ddot{x} + \frac{h^3}{6}\dddot{x} + \ldots
\end{align*}

Теперь, если мы повторим систему в общей сложности $N=\frac{t_N - t_0}{h}\propto h^{-1}$ раз, то общая глобальная ошибка составит $E = N\cdot e \propto h$. То есть, Эйлер - это метод первого порядка, как было заключено в предыдущих модулях. <strong>Теперь вопрос в следующем: можем ли мы получить лучшее приближенное решение с тем же размером шага?</strong>

## Совершенствование метода Эйлера

Основополагающая концепция метода Рунге-Кутты, которую мы хотим показать, построена на тех же основных понятиях, что и метод Эйлера. Теперь рассмотрим снова метод Эйлера, однако на этот раз мы будем использовать производную не в $x_{n+1}$, а в средней точке $x_{m}=\frac{x_{n+1}+x_n}{2}$, называемой *тестовой точкой*. Затем можно вычислить более точный $x_{n+1}$, используя информацию производной в этой средней точке ($\dot{x}_m$) интервала $h$ между $x_n$ и $x_{n+1}$. То есть,

\begin{align*}
x_{n+1} &= x_n + h\cdot \dot{x}_m \\[1.2em]
        &= x_n + h\cdot [\cos(x_m) + \sin(t_m)] \\[0.8em]
        &= x_n + h\cdot \left[\cos\left(x_n + \frac{h \dot{x}_n}{2}\right) + \sin\left(t_n+\frac{h}{2} \right)\right] \\[1.0em]
        &= x_n + h\cdot \left[\cos\left(x_n + \frac{h}{2}[\cos(x_n)+\sin(t_n)]\right) + \sin\left(t_n+\frac{h}{2} \right)\right]
\end{align*}

Теперь давайте сопоставим с исходным подходом Эйлера и изучим разницу.

In [ ]:
x_imp = np.zeros(N) 

for n in range(0,N-1):
    x_imp[n+1] = x_imp[n] + h * ( np.cos(x_imp[n]+(h/2.0)*(np.cos(x_imp[n]) + np.sin(t[n]))) + np.sin(t[n]+h/2.0)  )

plt.figure(figsize=(10,6))
plt.plot(t,x_Eu ,'-ro' ,linewidth=3.0,label=r'Euler')
plt.plot(t,x_imp,'--go',linewidth=3.0,label=r'Improved Euler')
plt.ylabel(r'Dimensionless position, $x(t)$')
plt.xlabel(r'Dimensionless time, $t$')
plt.title(r'Stepsize, $N$ = %i' % N)
plt.legend(loc=4) # 'loc' sets the location of the legend-text in the plot 
plt.grid()
plt.show()

Новое приближение существенно отличается от решения, полученного с помощью оригинального подхода Эйлера (с использованием $N=15$). То, что мы реализовали здесь, часто называют <i>Улучшенным методом Эйлера</i> или <i>Методом средней точки</i> для ОДУ. Однако у подхода много имен, и мы будем называть его **Методом Рунге-Кутты второго порядка**.

Давайте вкратце сравним ошибки двух подходов, сначала оценив ошибку нашей улучшенной схемы аналогичным образом, как описано выше, путем разложения в ряд в $(t+h/2)$. Улучшенная схема применяет аппроксимацию

$$
x(t+h) \approx x(t) + h\dot{x}(t+h/2)
$$

где мы раскладываем $\dot{x}(t+h/2)$, что приводит к

$$
\dot{x}(t+h/2) = \dot{x}(t) + (h/2)\ddot{x} + \frac{(h/2)^2}{2}\dddot{x} + \mathcal{O}(h^3)
$$

Аналогично методу Эйлера, мы сравниваем нашу улучшенную схему с точным разложением Тейлора,

\begin{align*}
\text{Exact:} & \qquad x(t + h) =  x(t) + h\dot{x} + \frac{h^2}{2}\ddot{x} + \frac{h^3}{6}\dddot{x} + \ldots  \\[1.2em]
\text{Impr. E.:} & \qquad x(t + h) \approx  x(t) + h[\dot{x}(t) + (h/2)\ddot{x}]\\[1.2em]
\text{Error:} & \qquad e = \frac{h^3}{6}\dddot{x} + \mathcal{O}(h^4)
\end{align*}

Это подразумевает глобальную ошибку $E=N\cdot e \propto h^2$, которая соответствует методу точности второго порядка по $h$, что не так удивительно, учитывая название метода. Давайте спросим себя: можем ли мы сделать еще лучше?

## Общий Метод Рунге-Кутты (Четвертого Порядка)

Эйлер оценивал $x_{n+1}$ с помощью $x_n$ и $\dot{x}_n$. То, что мы только что сделали, было оценкой $x_{n+1}$ с использованием $x_n$ и производной промежуточной точки, а именно средней тестовой точки, $x_m$. Теперь, вот интересная идея: ***Можно ли использовать аналогичную тестовую точку, чтобы вычислить производную от фактической тестовой точки?*** Потратьте некоторое время, чтобы подумать о том, что подразумевает этот вопрос, прежде чем читать дальше.

Возможно, не столь новаторский ответ на вышеприведенный вопрос - **да**. На самом деле, мы можем сделать даже лучше: мы можем сделать тестовую точку для тестовой точки тестовой точки и так далее! Как оказалось, *Метод Рунге-Кутты четвертого порядка* использует три таких тестовых точки и является наиболее широко используемым методом Рунге-Кутты. Вы можете спросить, почему мы не используем пять, десять или даже больше тестовых точек, и ответ довольно прост: вычисление всех этих тестовых точек не является бесплатным с точки зрения вычислений, и выигрыш в точности быстро уменьшается за пределами четвертого порядка метода. То есть, если высокая точность имеет такое значение, что вам потребуется Рунге-Кутта десятого порядка, то вам лучше уменьшить размер шага $h$, чем увеличить порядок метода. 

Кроме того, существуют другие более сложные методы, которые могут быть как более быстрыми, так и более точными для эквивалентного выбора $h$, но, очевидно, могут быть намного сложнее в реализации. См., Например, *Экстраполяцию Ричардсона*, *метод Булирша-Стоера*, *Многоступенчатые методы, Многозначные методы* и *Методы предиктора-корректора*.

Тем не менее, теперь мы покажем общее выражение для произвольно упорядоченного метода Рунге-Кутты, прежде чем мы применим метод четвертого порядка к задаче, приведенной выше. Опять же, рассмотрим ОДУ, написанную в форме

$$
\dot{x}(t) = g(x(t),t)
$$

Тогда для **общего** $q$-упорядоченного метода Рунге-Кутты

\begin{align*}
k_1 &= h\cdot g(x_n, t_n) \\[1.0em]
k_2 &= h\cdot g(x_n + a_{2,1}k_1                                   , t+c_2 h ) \\[1.0em]
k_3 &= h\cdot g(x_n + a_{3,1}k_1 + a_{3,2}k_2                      , t+c_3 h ) \\[1.0em]
k_4 &= h\cdot g(x_n + a_{4,1}k_1 + a_{4,2}k_2 + a_{4,3}k_3         , t+c_4 h ) \\[1.0em]
    &\qquad \vdots   \\[1.0em]
k_q &= h\cdot g(x_n + [a_{q,1} k_1 + a_{q,2}k_2+\ldots+a_{q,q-1}k_{q-1}], t_n + c_q h )
\end{align*}

Такие, что,

\begin{equation*}
x_{n+1} = x_n + \sum_{i=1}^{q} b_i k_i
\end{equation*}


Схема, представленная сейчас в ее общем виде, имеет некоторые неопределенные коэффициенты: $a_{i,j}$, $b_{i}$ и $c_{i}$. Элементы $a_{i,j}$ обозначаются как матрица Рунге-Кутты, в то время как $b_i$ - это веса, а $c_i$ - узлы. Вывод этих (и коэффициентов для других подобных методов) может быть утомительной алгебраической задачей. Следовательно, такие коэффициенты обычно получаются из таблиц в литературе. Коэффициенты методов Рунге-Кутты можно найти с помощью таблиц (John C.) Butcher,

\begin{array}{ c|c c c }
  0   &         &&              &&& \\
  c_2 & a_{2,1} &&              &&& \\
  c_3 & a_{3,1} && a_{3,2} &&& \\
  \vdots &      &\ddots&         &&& \\
  c_q & a_{q,1} && a_{q,2} & \ldots & a_{q,q-1}&& \\
  \hline
      & b_1     && b_2     & \ldots & b_{q-1}  && b_q
\end{array}

Затем коэффициенты определяются путем требования, чтобы метод был *согласованным*. Последовательность численного (конечно-разностного) приближения относится к тому факту, что аппроксимируемая задача (уравнение) приближается к точной задаче в пределе размера шага, приближающегося к нулю. Для метода Рунге-Кутты это происходит, когда

\begin{equation*}
\sum_{j=1}^{l-1} a_{i,j} = c_i, \text{  for  } l\in 2,3,\ldots,q;
\end{equation*}

Может существовать несколько вариантов коэффициентов для порядка $q$, но мы не будем вдаваться в подробности их вывода. Вместо этого мы приводим, возможно, наиболее широко применяемый выбор для метода четвертого порядка $(q=4)$, для которого таблица Батчера имеет вид

\begin{array}{ c|c c c }
  0   &     &     &     &    \\
  1/2 & 1/2 &     &     &    \\
  1/2 & 0   & 1/2 &     &    \\
  1   & 0   & 0   & 1   &    \\
  \hline
      & 1/6 & 1/3 & 1/3 & 1/6 \\
\end{array}

Приведенная выше таблица позволяет вычислять $k_1$, $k_2$, $k_3$ и $k_4$ таким образом, что теперь мы можем применить метод Рунге-Кутты четвертого порядка к рассматриваемой задаче.

In [ ]:
x_4RK = np.zeros(N) ## Runge-Kutta

def g(x_,t_):
    return np.cos(x_) + np.sin(t_)

for n in range(0,N-1):
    k1 = h*g( x_4RK[n]       , t[n]         )
    k2 = h*g( x_4RK[n] + k1/2, t[n] + (h/2) ) 
    k3 = h*g( x_4RK[n] + k2/2, t[n] + (h/2) ) 
    k4 = h*g( x_4RK[n] + k3  , t[n] +  h    )
    
    x_4RK[n+1] = x_4RK[n] + k1/6 + k2/3 + k3/3 + k4/6
    
plt.figure(figsize=(10,6))
plt.plot(t,x_Eu, '-ro' , linewidth=3.0,label=r'Euler')
plt.plot(t,x_imp,'--go', linewidth=3.0,label=r'2nd order Runge-Kutta')
plt.plot(t,x_4RK,':bo' , linewidth=3.0,label=r'4th order Runge-Kutta')
plt.ylabel(r'Dimensionless position, $x(t)$')
plt.xlabel(r'Dimensionless time, $t$')
plt.title(r'Stepsize, $N$ = %i' % N)
plt.legend(loc=4) 
plt.show()

Видно, что существует незначительная разница между реализациями Рунге-Кутты четвертого и второго порядка, в то время как метод Эйлера имеет значительную ошибку. Мы рекомендуем вам скорректировать общее количество точек $N$, чтобы увидеть, как различные методы соотносятся друг с другом $(h \overset{N \to \infty}{\longrightarrow} 0)$.

### Производные более высокого порядка и наборы ОДУ 1-го порядка

До сих пор мы ограничивались рассмотрением ОДУ, содержащих только производные первого порядка, и в принципе, этого достаточно. Будьте осторожны, чтобы не перепутать порядок (производной) уравнения и порядок (точности) численного метода. ОДУ порядка $q$, всегда можно свести к набору из двух ОДУ: первого и порядка $(q-1). Рассмотрим общую линейную ОДУ второго порядка с постоянными коэффициентами и некоторыми произвольными начальными условиями

$$
a\ddot{x} + b\dot{x} + cx = g(t), \qquad x = x(t)
$$

Затем введем новую переменную $\nu(t) \equiv \dot{x}(t)$, такую, что описанная выше проблема может быть выражена

\begin{align*}
\dot{x}   &= \nu                            &\equiv F(x,\nu,t)\\[1.0em]
\dot{\nu} &= \frac{1}{a} ( g(t) - b\nu - cx ) &\equiv G(x,\nu,t)
\end{align*}

Где мы ввели $F,G$ только для общности. На самом деле этот метод был бы применим и для более общего случая,когда $a,b, c$ являются функциями $x,\nu,t$.

Тем не менее, этот набор из двух зависимых ОДУ первого порядка может быть решен с помощью метода Рунге-Кутты! На самом деле, мы можем решить любой набор ОДУ первого порядка, однако имейте в виду, что в этом конкретном примере тестовые точки для $x$ будут зависеть от тестовых точек для $\nu$ и наоборот, так что два уравнения должны быть решены одновременно. Это также относится к набору ОДУ произвольного размера.

Для метода Рунге-Кутты четвертого порядка получаем для временного шага $n$

\begin{align*}
k_{x1} &= h \cdot F\left(x_n                   ,\nu_n                     ,t_n             \right), & k_{\nu1} &= h \cdot G\left(x_n                   ,\nu_n                     ,t_n             \right) \\[1.0em]
k_{x2} &= h \cdot F\left(x_n + \frac{k_{x1}}{2},\nu_n + \frac{k_{\nu1}}{2},t_n+\frac{h}{2} \right), & k_{\nu2} &= h \cdot G\left(x_n + \frac{k_{x1}}{2},\nu_n + \frac{k_{\nu1}}{2},t_n+\frac{h}{2} \right) \\[1.0em]
k_{x3} &= h \cdot F\left(x_n + \frac{k_{x2}}{2},\nu_n + \frac{k_{\nu2}}{2},t_n+\frac{h}{2} \right), & k_{\nu3} &= h \cdot G\left(x_n + \frac{k_{x2}}{2},\nu_n + \frac{k_{\nu2}}{2},t_n+\frac{h}{2} \right) \\[1.0em]
k_{x4} &= h \cdot F\left(x_n +       k_{x3}    ,\nu_n +       k_{\nu3}    ,t_n+h           \right), & k_{\nu3} &= h \cdot G\left(x_n +       k_{x3}    ,\nu_n +       k_{\nu3}    ,t_n+h           \right)
\end{align*}

Такие, что,

\begin{align*}
    x_{n+1}   &=   x_n + k_{x1}/6    + k_{x2}/3    + k_{x3}/3    + k_{x4}/6    \\[1.0em]
    \nu_{n+1} &= \nu_n + k_{\nu 1}/6 + k_{\nu 2}/3 + k_{\nu 3}/3 + k_{\nu 4}/6 
\end{align*}


Для $n=1,\ldots,N-1;$ в то время как $x_0$, $\nu_0$ должны быть известны как начальные условия. Обратите внимание, что размер шага $h$ может быть выведен из $k$ и оставлен для вычисления $x_{n+1}$ вместо этого, что несколько уменьшит общее количество вычислений/операций в каждом цикле. Вы также должны заметить, что $k_{xi}$ зависит от $k_{\nu (i-1)}$ и $k_{\nu i}$ зависит от $k_{x(i-1)}$. Таким образом, значения $k$ должны быть рассчитаны упорядоченным образом!

Мы приходим к выводу, что для системы $M$ уравнений первого порядка общее число $k$-значений для $q$-упорядоченного метода Рунге-Кутты будет равно $M \cdot q$. То есть большие наборы дифференциальных уравнений с производными более высоких порядков приведут к еще большему набору ОДУ первого порядка, дающих огромное количество $k$-переменных. Если это так, вы, возможно, захотите пересмотреть, действительно ли метод Рунге-Кутты является лучшим подходом к вашей проблеме.

Тем не менее, метод Рунге-Кутты довольно прост в реализации, и обеспечивает то, что мы будем называть довольно приличной точностью, в то же время все еще достаточно быстр и эффективен. Это, безусловно, мощный инструмент для любого ученого занимающегося вычислениями!